In [ ]:
%load_ext autoreload

%autoreload 2


In [ ]:
import os,sys
os.environ['BASE_DIR'] =  '/Users/macico/Dropbox/btc'
os.environ['KULOKO_DIR'] = os.path.join(os.environ['BASE_DIR'], "kuloko")
os.environ['ALEISTER_DIR'] = os.path.join(os.environ['BASE_DIR'], "aleister")
os.environ['COMMON_DIR'] = os.path.join(os.environ['BASE_DIR'], "geco_commons")
os.environ['KULOKO_INI'] = os.path.join(os.environ['COMMON_DIR'], "ini")
os.environ['ALEISTER_INI'] = os.path.join(os.environ['COMMON_DIR'], "ini")
os.environ['MONGO_DIR'] = os.path.join(os.environ['COMMON_DIR'] ,"mongodb")
os.environ['LOGDIR'] = os.path.join(os.environ['KULOKO_DIR'], "log")
# os.environ['MLFLOW_TRACKING_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
# OS.environ['MLFLOW_ARTIFACTS_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
os.environ['MLFLOW_ARTIFACTS_URI'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
os.environ['DEFAULT_LOCAL_FILE_AND_ARTIFACT_PATH'] = os.path.join(os.environ['BASE_DIR'], "mlruns")
sys.path.append(os.path.join(os.environ['KULOKO_DIR'],"items" ))


In [ ]:


import master
from master import *
from datetime import datetime as dt

In [ ]:


#meta
_id = "test002"
config_mode = "DEV"
model_name = "SLSTM"


# sym
sym = "BTC"

# load all parent modules
fp = featurePreprocess(_id)
fp.load_general_config(source="ini", path=None,mode=config_mode)
fp.load_model_config(source="ini", path=None,model_name=model_name)

In [ ]:

dg =  DataGen()
dg.get_load_data_proxy()
fetch_start = 20210201
fetch_end = 20210228
train_start = 20210201
train_end = 20210208
valid_start = 20210209
valid_end = 20210215
test_start = 20210216
test_end = 20210222
trades = dg.get_trades(sym=sym, sd=fetch_start, ed=fetch_end)
fp._logger.info("[DONE] Get prepro raw data. {0}~{1}".format(fetch_start,fetch_end))
Xy = dg.get_Xy(trades)

In [ ]:
Xy 

In [ ]:
# prepro
ans_col = fp.model_config.get("ANS_COL")
X_train, X_val, X_test, y_train, y_val, y_test = \
    fp.convert_dataset(Xy, ans_col, "period", 
                        train_start=train_start, train_end=train_end, 
                        valid_start=valid_start, valid_end=valid_end, 
                        test_start=test_start, test_end=test_end)
# X_train, X_val, X_test, y_train, y_val, y_test = fp.convert_dataset(Xy, ans_col,split_rule="period", test_ratio=0.4, valid_ratio=0.5)
scaler  = fp.get_scaler("minmax")
X_train,x_scaler = fp.scalingX(X_train)
X_val,_ = fp.scalingX(X_val,x_scaler)
X_test,_ = fp.scalingX(X_test,x_scaler)

In [ ]:
import torch

from custom_dataloader import SequenceDataset
X_trains = [X_train]
train_features = [ torch.Tensor(X_train) for X_train in X_trains] 
train_targets = torch.Tensor(y_train)

# make dataset 
train = SequenceDataset(*train_features, train_targets, **{"window_size":10,"batch_size":2})

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_loader = DataLoader(train, batch_size=64, shuffle=False, drop_last=True)

In [ ]:
for a,b in  train_loader:
    print(a)
    break

In [ ]:
a = np.array([0.09725063,0.10534658,0.10118312,0.10872556,0.8832147,0.2715066,
 0.06436768])

In [ ]:

np.repeat(train.Xs[0][0][None,:], 0, axis=0)

In [ ]:
train.Xs[0][0]
